In [1]:
import pandas as pd
# The sqlite3 module provides a Python interface for SQLite databases.
import sqlite3 as sql
import re

## SQL

In [2]:
# 下载下来到本地方便些，要不代码看起来怪怪的
# remote_database_url = 'your_remote_database_url'
# conn = sql.connect(f"jdbc:sqlite:{remote_database_url}")
conn = sql.connect(r"C:\Users\catri\OneDrive\文档\LIWU\HW5\user_actions.db")

# a cursor is a database object that facilitates the interaction with the database by allowing the execution of SQL queries, 
#fetching results, and providing a mechanism for navigation and manipulation of data. 要转成cursor才能用SQL的命令
cur = conn.cursor()

In [3]:
# Query to retrieve the list of tables
tables = list(cur.execute("SELECT name FROM sqlite_master WHERE type='table';"))

tables

schema = list(cur.execute("SELECT name, sql FROM sqlite_master WHERE type='table';"))
schema


[('user_actions',
  'CREATE TABLE user_actions (\n    user_id INTEGER,\n    username TEXT,\n    email TEXT,\n    action TEXT,\n    timestamp TEXT\n)')]

In [4]:
#连接完数据库关掉很重要，1是保护资源，2接口有时候是有限制的，
query = "SELECT * FROM user_actions;"
user_actions = list(cur.execute(query))

In [5]:
user_actions_df = pd.DataFrame(user_actions)
user_actions_df.head()

,0,1,2,3,4
0,34,user34,user34@email.com,signup,2015-02-04 14:38:47
1,28,user28,user28@email.com,signup,2015-03-09 11:55:33
2,27,user27,user27@email.com,login,2015-04-17 14:48:31
3,27,user27,user27@email.com,login,2015-04-21 13:22:14
4,27,user27,user27@email.com,reset_password,2015-04-25 16:30:15


In [6]:
#Retrieve the usernames of all users who have performed the "signup" action.

query = "SELECT username FROM user_actions WHERE action = 'signup';"#signup那里之前用双引号就不行。
user_name_signup = list(cur.execute(query))
#user_name_signup

In [7]:
# Find the total number of log entries(登录的次数，我理解) for each user. Display the user_id, username, and the count of log entries.
query = "SELECT user_id, username,COUNT(*) AS total_entries FROM user_actions GROUP BY user_id;" #这里为什么名字没打印出来，AS 在这里没用到
user_name_logEntries = list(cur.execute(query))
#user_name_logEntries

In [8]:
# Identify users who have both logged in (action = 'login') and signed up (action = 'signup') on the same day. 
# Display the user_id and username.这个要好好总结，不是GDP完全不会。拆分下一个一个总结
query = "SELECT user_id, username FROM user_actions WHERE action IN ('login', 'signup') GROUP BY user_id, username, DATE(timestamp) HAVING COUNT(DISTINCT action) = 2 ;" 
query3 = list(cur.execute(query))
#query3

## Regex

In [33]:
#Write a regular expression to extract all hashtags from a given comment. 
#For example, applying the regex to comment 1 should return ["#programming", "#tips"].

textfile = open("comments.txt", 'r')
matches_flat = []
matches_dict = {}
reg = re.compile(r'(#[\w]+)[\s|"]')
for line_num, line in enumerate(textfile):
    matches = reg.findall(line)
    matches_flat += matches
    matches_dict[line_num] = matches
    line_num += 1
print(f">>>All hashtags: {matches_flat}")

textfile.seek(0)
reg2 = re.compile(r'^(?=.*\#programming\b)(?=.*\#python\b).*$')
print(">>>Comments contain both #programming and #python is", end = " ")
for line_num, line in enumerate(textfile):
    line_num += 1
    matches = reg2.findall(line)
    if matches:
        print(line_num, end= " ")

textfile.seek(0)
line2 = textfile.readlines()[1]
print(f"\n>>>2nd comment {'' if reg2.findall(line2) else 'does not'} contains both #programming and #python")


#print(f">>>Uniq hastags: {list(set(matches_flat))}")

>>>All hashtags: ['#programming', '#tips', '#coding', '#python', '#tech', '#data', '#programming', '#coding', '#data', '#analysis', '#programming', '#python', '#tips', '#tech', '#innovation', '#coding', '#python', '#data', '#analytics', '#programming', '#tips', '#coding', '#python', '#tech', '#data', '#programming', '#coding', '#data', '#insights', '#programming', '#python', '#tips', '#tech', '#research', '#coding', '#python', '#data', '#analytics', '#programming', '#tips', '#coding', '#innovation', '#programming', '#python', '#tips', '#tech', '#data', '#coding', '#python', '#data', '#analytics', '#programming', '#tips', '#coding', '#python', '#tech', '#innovation', '#programming', '#coding', '#data', '#analytics', '#programming', '#python', '#tips', '#coding', '#innovation', '#tech', '#data', '#programming', '#tips', '#coding', '#python', '#data', '#analysis', '#programming', '#tips', '#coding', '#python', '#tech', '#innovation', '#programming', '#coding', '#data', '#analytics', '#pro